In [1]:
import os

In [2]:
os.getcwd()

'/Users/mac/Desktop/Kidney-Disease_Classifier--DeepLearning Project/research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'/Users/mac/Desktop/Kidney-Disease_Classifier--DeepLearning Project'

In [5]:

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Harry-Potter20/Kidney-Disease-Classifier---DeepLearning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Harry-Potter20"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d094eb8680b50e84269fa4303f872962deca1c15"

In [6]:
import tensorflow as tf

2025-04-10 00:04:43.548185: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model="artifacts/training/model.h5", 
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone", 
            mlflow_uri="https://dagshub.com/Harry-Potter20/Kidney-Disease-Classifier---DeepLearning-Project.mlflow",  
            all_params=self.params, 
            params_image_size=self.params.IMAGE_SIZE, 
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bicubic"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the model Registry, which depends on the use case,
                # please refer to the docs for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow

                mlflow.keras.log_model(self.model, "model", registered_model_name="ResNet101V2")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-04-10 00:04:57,031: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-10 00:04:57,073: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-10 00:04:57,075: INFO: common: created directory at: artifacts]
Found 2487 images belonging to 4 classes.


2025-04-10 00:05:04.591478: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


156/156 [==============================] - 539s 3s/step - loss: 31.5464 - accuracy: 0.6321
[2025-04-10 00:14:03,678: INFO: common: json file saved at: (path)]


2025/04/10 00:14:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-04-10 00:14:41,963: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 105). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/h8/19h5sq255tvd7309yghg4ycr0000gn/T/tmp7e9bwksb/model/data/model/assets
[2025-04-10 00:14:55,739: INFO: builder_impl: Assets written to: /var/folders/h8/19h5sq255tvd7309yghg4ycr0000gn/T/tmp7e9bwksb/model/data/model/assets]


Registered model 'ResNet101V2' already exists. Creating a new version of this model...
2025/04/10 00:16:09 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ResNet101V2, version 6
Created version '6' of model 'ResNet101V2'.
